# Google Cloud with langchain


<h3>Tracking LangChain Executions with Aim</h3>

In this notebook we will explore three usage scenarios. To start off, we will install the necessary packages and import certain modules. Subsequently, we will configure two environment variables that can be established either within the Python script or through the terminal.

In [ ]:
%pip install -U aim
%pip install -U langchain
%pip install -U VertexAI
%pip install -U google-search-results
%pip install -U google-cloud-secret-manager==2.16.1 google-crc32c==1.5.0
%pip install -U wikipedia

In [ ]:
PROJECT_ID=!gcloud config get-value project

print(f"PROJECT_ID: {PROJECT_ID}")

In [7]:
import os
from datetime import datetime

from langchain.llms import VertexAI
from langchain.callbacks import AimCallbackHandler, StdOutCallbackHandler

In [ ]:
from google.cloud import secretmanager
import google_crc32c


def access_secret_version(
    project_id: str, secret_id: str, version_id: str
) -> str:
    """
    Access the payload for the given secret version if one exists. The version
    can be a version number as a string (e.g. "5") or an alias (e.g. "latest").
    """
    VERBOSE = False
    
    # Import the Secret Manager client library.
    from google.cloud import secretmanager

    # Create the Secret Manager client.
    client = secretmanager.SecretManagerServiceClient()

    # Build the resource name of the secret version.
    name = f"projects/{project_id}/secrets/{secret_id}/versions/{version_id}"

    # Access the secret version.
    response = client.access_secret_version(request={"name": name})

    # Verify payload checksum.
    crc32c = google_crc32c.Checksum()
    crc32c.update(response.payload.data)
    if response.payload.data_crc32c != int(crc32c.hexdigest(), 16):
        print("Data corruption detected.")
        return response

    # Print the secret payload.
    #
    # WARNING: Do not print the secret in a production environment - this
    # snippet is showing how to access the secret material.
    payload = response.payload.data.decode("UTF-8")
    if VERBOSE == True:
        print(f"Plaintext: {payload}")
    
    return payload

print(PROJECT_ID)

CSE_ID = access_secret_version(PROJECT_ID, "CSE_ID", "latest")
GOOGLE_API_KEY = access_secret_version(PROJECT_ID, "GOOGLE_API_KEY", "latest")


In [19]:
import os

os.environ["GOOGLE_API_KEY"]= GOOGLE_API_KEY
os.environ["GPLACES_API_KEY"] = GOOGLE_API_KEY
os.environ["GOOGLE_CSE_ID"] = CSE_ID

The event methods of `AimCallbackHandler` accept the LangChain module or agent as input and log at least the prompts and generated results, as well as the serialized version of the LangChain module, to the designated Aim run.

In [ ]:
session_group = datetime.now().strftime("%m.%d.%Y_%H.%M.%S")
aim_callback = AimCallbackHandler(
    repo=".",
    experiment_name="scenario 1:  LLM",
)

callbacks = [StdOutCallbackHandler(), aim_callback]
llm = VertexAI( callbacks=callbacks)

The `flush_tracker` function is used to record LangChain assets on Aim. By default, the session is reset rather than being terminated outright.

<h3>Scenario 1</h3> In the first scenario, we will use VertexAI LLM.

In [24]:
# scenario 1 - VertexAI LLM
llm_result = llm.generate(["Tell me a joke", "Tell me a poem"] * 3)
aim_callback.flush_tracker(
    langchain_asset=llm,
    experiment_name="scenario 1: Chain with multiple SubChains on multiple generations",
)

llm_result.generations

[[Generation(text='What do you call a fish with no eyes? Fsh!', generation_info=None)],
 [Generation(text="**The World is a Beautiful Place**\n\nThe world is a beautiful place,\nFull of wonder and grace.\nFrom the mountains to the sea,\nThere's beauty for all to see.\n\nThe trees stand tall and proud,\nTheir leaves a verdant shroud.\nThe flowers bloom in the sun,\nTheir petals a rainbow of fun.\n\nThe animals roam free,\nIn harmony with the sea.\nThe birds sing in the trees,\nTheir songs a symphony of peace.\n\nThe world is a beautiful place,\nFull of wonder and grace.\nLet us all take a moment", generation_info=None)],
 [Generation(text='What do you call a fish with no eyes? Fsh!', generation_info=None)],
 [Generation(text="**The World is a Beautiful Place**\n\nThe world is a beautiful place,\nFull of wonder and grace.\nFrom the mountains to the sea,\nThere's beauty for all to see.\n\nThe trees stand tall and proud,\nTheir leaves a verdant shroud.\nThe flowers bloom in the sun,\nTheir

<h3>Scenario 2</h3> The third scenario involves an agent with tools.

In [26]:
from langchain.agents import initialize_agent, load_tools
from langchain.agents import AgentType

In [ ]:
from langchain.tools import GooglePlacesTool

places = GooglePlacesTool()

In [36]:
# scenario 2 - Agent with Tools
tools_std = load_tools(["google-search",  "wikipedia", "llm-math"], llm=llm, callbacks=callbacks)

tools = [places]+tools_std


agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    callbacks=callbacks,
)
agent.run(
    "What is the more expensive restaurant in the birth place of Zinedine zidane ? "
)
aim_callback.flush_tracker(langchain_asset=agent, reset=False, finish=True)




> Entering new  chain...
I need to know the birth place of Zinedine zidane 
Action: google_search
Action Input: Zinedine zidane birth place
Observation: Zidane is of Algerian Kabyle descent and has described himself as a "non-practising Muslim". His parents, Smaïl and Malika, immigrated to Paris from the village ... 6 days ago ... Zinedine Zidane, byname Zizou, (born June 23, 1972, Marseille, France), French football (soccer) player who led his country to victories in ... Zidane, of Algerian origin, was raised in Marseille. He began playing with the youth teams of his native city, from where he went to Cannes, with whom he made ... Apr 2, 2014 ... Zinedine Yazid Zidane was born on June 23, 1972, in Marseille, France. The son of Algerian immigrants, Zidane learned to play soccer in the ... Zinedine Zidane was born in Marseille, France, on June 23, 1972. At the tender age of five, young Zidane was introduced to the football game. Together with his ... Personal information ; Full name, 